## Introduction and references

In this blog article, I present a brief overview on the use of quaternions
for ridid-body dynamics in Molecular Dynamics. This topic is quite old and
is found in many reference material. For some reason, none of the presentations
seemed to me to provide a brief presentation of the quaternions, their relation
to the equations of motion of a rigid body and how they can used in Molecular Dynamics.
This is probably more of problem on my side than one with the material but as I prepare
these notes I might as well publish them here.

The references mentioned in the text are:
- [Goldstein2002]: Goldstein, Poole and Safko, *Classical Mechanics* $3^\mathrm{rd}$ edition (Pearson Education, San Francisco, CA, 2002).
- [Rapaport1995]: Rapaport, *The art of molecular dynamics simulation* (Cambridge University Press, 1995).


## Notation and definition of the quaternions

The notation is set to "completely abstract". Any quantity can a scalar, vector
or quaternion depending on the context. For convenience, though, the following letters
are reserved for specific quantities:

- $r$ is a vector denoting the position of a point or particle in Euclidean space.
- $r_i$ is as $r$, but for the $i^\mathrm{th}$ particle of the rigid body.
- $v_i$ is the velocity of a particle.
- $\omega$ is the angular velocity of the rigid body.
- $I$ is the inertia tensor, with components $I_{i,j}$.
- $L$ is the angular momentum, a vector.
- $\mathcal{T}$ is the torque on the rigid body.
- $q$, $q_1$, etc. are quaternions.
- The superscript $^B$ is for quantities defined in the frame of reference of the rigid body. Other quantities are in the laboratory reference frame.

When an operation involves quaternions and vectors, the result is truncated to "vector" only
implicitly when needed (i.e. when the result is a physical quantity).

Quaternions are defined as the sum of a scalar part $s$ and a 3-dimensional vector part $v$ as

$$q = s + v$$

Multiplication of the elementary vectors $v_i=(1, 0, 0)$, $v_j=(0, 1, 0)$ and $v_k=(0,0,1)$:
- $v_i v_j = v_k$
- $v_j v_k = v_i$
- $v_k v_i = v_j$
- $v_j v_i = -v_k$
- $v_k v_j = -v_i$
- $v_i v_k = -v_j$
- $v_i^2 = v_j^2 = v_k^2 = -1$

## Quaternions for rigid-body Molecular Dynamics

The motion of a rigid body can be defined entirely on the basis of the
rotation operator that transforms vectors from the rigid body reference frame to the
laboratory reference frame. Explicitly, any point of particle with coordinates $r^B$
can be transformed as

$$r = R r^B$$

Using quaternions, the relation is written

$$r = q r^B q^\ast$$

and obtaining the time-dependent quaternion $q(t)$ provides a solution to this problem.

The starting point is the equations of motion for a rigid body:
$$\dot L = \mathcal{T}$$

or otherwise

$$ \dot {\left( I \omega \right)} = \mathcal{T}$$

so that to obtain the time derivative of $\omega$, one has to invert $I$.
The solution is to work in the so-called principal axes frame of reference that
is intrinsic to the rotating body.

$$ I^B \dot \omega^B = \mathcal{T^B} + \omega \wedge \left( I^B \omega^B \right)$$


First, a few definitions to settle things down

$\omega$ is the angular velocity in the laboratory reference frame. It is defined as

$$\omega = \frac{r\wedge v}{|r|^2}$$
where $r$ and $v$ are the position and velocity of any particle in the rigid body.

Another relation gives the velocity of any point in the rigid body as a function of its position:

$$v = \omega \wedge r$$

The angular momentum is defined as

$$L = I \omega$$

where $I$ is a 3 by 3 matrix called the "tensor of inertia", whose elements are defined by

$$I_{ij} = \sum_k m_k \left( r_k^2 \delta_{ij} - r_{k,i} r_{k,j} \right)$$

where $m_k$ is the mass of the $k^\mathrm{th}$ particle, $r_i$ its vector position and $r_{k,i}$ the $i^\mathrm{th}$ component of the vector.

The equation of motion for $I$ is


In this reference frame, $I^{B}$ is diagonal (the $B$ superscript stands for "Body").

$$I^B = \left(\begin{matrix}I^B_x & 0 & 0\cr 0 & I^B_y & 0\cr 0 & 0 & I^B_z\end{matrix}\right)$$

$$L^B = I^B \omega^B$$

and

$$\dot L^B = \dot {\left( I^B \omega^B \right)}$$

The equations of motion in the body reference frame requires the torque in this same reference frame.

### Rotations

The interest of quaternions arises from their capacity to represent rotations
in 3 dimensions as:

$$r' = q r q^\ast$$

For rigid-body dynamics, the problem can be recast in the equations of motion
for $q$.

The position $r$ is related to the body-frame position $r^B$ as

$$r(t) = q(t) r^B q^\ast(t)$$

where $r^B$ is fixed in time. The explicit dependence on $t$ is dropped in the notation
from now on.

The equations of motion can be cast as

$$\dot r = \dot q r^B q^\ast + q r^B \dot q^\ast$$

In [ ]:
import sympy
from sympy import *
init_session()

In [ ]:
class quat(object):
    def __init__(self, qvec, q4):
        self.qvec = sympy.Matrix(qvec)
        self.q4 = q4
    def __mul__(self, qq):
        if isinstance(qq, Matrix):
            qq = quat(qq, 0)
        return quat(self.q4*qq.qvec + self.qvec*qq.q4 + self.qvec.cross(qq.qvec), self.q4*qq.q4-self.qvec.dot(qq.qvec))
    

In [ ]:
q1 = quat((1,0,0), 0)
q2 = quat((0,1,0), 0)
one_x = Matrix([1,0,0])

In [ ]:
q3 = q1*one_x

In [ ]:
q3.qvec, q3.q4

In [ ]:
def qmul(q1, q2):
    return ()

In [ ]:
import transforms3d

In [ ]:
help(transforms3d.quaternions)

In [ ]:
q = np.array([0.0, 1, 0, 0], dtype=float)
q /= np.sqrt(np.sum(q**2))

In [ ]:
qmult(q, qmult(q, qinverse(q)))

In [ ]:
np.dot(transforms3d.quaternions.quat2mat(q), q[1:])

In [ ]:
transforms3d.quaternions.qmult([0,1,0,0], [0,1,0,0])

In [ ]:
from transforms3d.quaternions import qmult, qinverse, qnorm

In [ ]:

q = [np.cos(2*np.pi/3), np.sin(2*np.pi/3)/np.sqrt(3), np.sin(2*np.pi/3)/np.sqrt(3), np.sin(2*np.pi/3)/np.sqrt(3)]

In [ ]:
qmult(q, qmult([0, 0, 0, 1], qinverse(q)))

In [ ]:
q

In [ ]:
qnorm(q)